In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_Iowa', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.shape

(591790, 54)

In [4]:
df.head()

,cid,createDate,createBy,dataFrom,X,Y,OBJECTID,CRASH_KEY,CASENUMBER,LECASENUM,...,YCOORD,REST_UPDATED,REST_UPDATE_UTC_OFFSET,CRASH_DATETIME,CRASH_DATETIME_UTC,CRASH_DATETIME_UTC_OFFSET,CITY_NAME,COUNTY_NAME,CARDINAL,GLOBALID
0,1000000002,2020-03-06 09:40:05,David,Iowa,-93.643864403392413,41.608562255863369,2182565,2009015329,2009502656,20090005774,...,4606517,2020-02-20T16:00:13.245Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f83adca9-a3c5-42a7-ab28-58a91380c99d
1,1000000003,2020-03-06 09:40:05,David,Iowa,-93.696177565791032,41.633045134955132,2182566,2009015332,2009502660,20090010547,...,4609269,2020-02-20T16:00:13.245Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cd525cb4-bf39-40c2-8a8d-e6fa1cbd22e7
2,1000000004,2020-03-06 09:40:05,David,Iowa,-93.658429392761718,41.602201728924989,2182567,2009015334,2009502662,20090010676,...,4605820,2020-02-20T16:00:13.245Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6ac289cc-2a8f-4e52-88b5-24e28a5f9888
3,1000000005,2020-03-06 09:40:05,David,Iowa,-95.805733265575,41.671057804625029,2182568,2009015974,2009503411,59-2009,...,4617059,2020-02-20T16:00:13.245Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5a67a5de-b30c-4f5c-8e18-a2acab4dbed3
4,1000000006,2020-03-06 09:40:05,David,Iowa,-91.534557801945141,41.665252714843014,2182569,2009015982,2009503421,2009-19611,...,4613648,2020-02-20T16:00:13.245Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d7e9c468-9e2b-4cc4-a5d7-cbfb6893be3d


In [5]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'X', 'Y', 'OBJECTID',
       'CRASH_KEY', 'CASENUMBER', 'LECASENUM', 'CRASH_DATE', 'CRASH_MONTH',
       'CRASH_DAY', 'TIMESTR', 'DISTRICT', 'COUNTY_NUMBER', 'CITY_NUMBER',
       'SYSTEMSTR', 'LITERAL', 'FRSTHARM', 'LOCFSTHRM', 'CRCOMNNR', 'MAJCSE',
       'DRUGALC', 'ECNTCRC', 'LIGHT', 'CSRFCND', 'WEATHER', 'RCNTCRC', 'RDTYP',
       'PAVED', 'WZRELATED', 'CSEV', 'FATALITIES', 'INJURIES', 'MAJINJURY',
       'MININJURY', 'POSSINJURY', 'UNKINJURY', 'PROPDMG', 'VEHICLES',
       'TOCCUPANTS', 'REPORT', 'XCOORD', 'YCOORD', 'REST_UPDATED',
       'REST_UPDATE_UTC_OFFSET', 'CRASH_DATETIME', 'CRASH_DATETIME_UTC',
       'CRASH_DATETIME_UTC_OFFSET', 'CITY_NAME', 'COUNTY_NAME', 'CARDINAL',
       'GLOBALID'],
      dtype='object')

In [6]:
df2 = pd.DataFrame()
df2['cid'] = df.cid
df2['city'] = df.dataFrom
df2['crash_time'] = pd.to_datetime(df.CRASH_DATE)
df2['injured_count'] = df.FATALITIES.astype('int') + df.INJURIES.astype('int') + df.MAJINJURY.astype('int') + df.MININJURY.astype('int')
df2['latitude'] = df.Y
df2['longitude'] = df.X
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [7]:
df2.head()

,cid,city,crash_time,injured_count,latitude,longitude,crash_date
0,1000000002,Iowa,2009-02-22 00:00:00+00:00,0,41.608562255863369,-93.643864403392413,2009-02-22
1,1000000003,Iowa,2009-04-06 00:00:00+00:00,2,41.633045134955132,-93.696177565791032,2009-04-06
2,1000000004,Iowa,2009-04-07 00:00:00+00:00,1,41.602201728924989,-93.658429392761718,2009-04-07
3,1000000005,Iowa,2009-04-19 00:00:00+00:00,2,41.671057804625029,-95.805733265575,2009-04-19
4,1000000006,Iowa,2009-04-17 00:00:00+00:00,0,41.665252714843014,-91.534557801945141,2009-04-17


In [8]:
df2.to_sql('cleaned_crash_Iowa',con = db_connection, if_exists="replace",index=False)

In [ ]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)